In this notebook, we answer [a "Riddler Classic" question posed](https://fivethirtyeight.com/features/can-you-solve-the-chess-mystery/) in The Riddler section of  [FiveThirtyEight](https://fivethirtyeight.com)

The question concerns John Conway's Game of Life, originally made popular by a [Mathematical Games column](https://web.stanford.edu/class/sts145/Library/life.pdf) by Martin Gardner.

The problem, as proposed in The Riddler, is:

>Now suppose we were to replace the infinite grid with a finite grid that has periodic boundary conditions, so that cells in the first row are neighbors with cells in the last row, and cells in the first column are neighbors with cells in the last column. If there are three rows and N columns, what is the smallest value of N that can support an oscillator?

Here, we present a computational solution to this problem. We investigate all possible initial configurations and follow their evolution, to identify periodicities.

In a finite board with periodic boundaries, the total number of possible configurations is finite, so all evolutions are *eventually periodic*, that is, after a finite number of steps it settles down in a periodic pattern, as illustrated below:

<img src="life-evolution.png" width="50%" height="50%">

We call the initial segment $C_0,C_1,\ldots,C_{t-1}$ the *transient part* and the final cycle $P_0,P_2,\ldots,P_{k-1}$ the *periodic part* of the evolution. In the picture above, the length of the transient part is $t$ and the length of the periodic part is $k$. The length of the periodic part is called the *eventual period* of the evolution.

There are two possibilities for this periodic pattern:

- It can consist of a state that just repeats itself indefinitely. For example, and empty board will remain empty forever. Borrowing terminology from Markov chains, we call these states *absorbing*. In other words, an absorbing state is symply a "cycle" of length 1.
- Alternatively, the evolution may settle down on a cycle of period two or more, which we call an *oscillator*.

This suggests the following algorithm to search for oscillators:

1. For each possible initial configuration $C_0$, do the following:
2. Compute the successive iterations $C_1,C_2,\ldots$ until a periodicity is detected.
3. If the period of the tail is two or more, we have found an oscillator.

Since the number of possible initial configurations for a $3\times n$ board is $2^{3n}$, it is only possible to carry out the computations for small values of $n$. We hope that we can find an oscillator with a small enough value of $n$.

Let's now get to the code. To represent a Game of Life configuration, we define a Python class called `life_pattern`, defined in the cell below.

In [8]:
def tuplefy_rectangle(rect):
    return tuple([tuple(row) for row in rect])

class life_pattern(object):
    directions = ((1,0), (1,1), (0,1), (-1,1), (-1,0), (-1,-1), (0,-1), (1,-1))
    
    def __init__(self, config):
        self._config = tuplefy_rectangle(config)
        self._height = len(config)
        self._width = len(config[0])

    def __str__(self):
        return str(self.config)

    def __repr__(self):
        return 'life_pattern(config={})'.format(self._config)
    
    def __eq__(self, other):
        return self._config == other._config
    
    def __ne__(self, other):
        return self._config != other._config
    
    def __hash__(self):
        return hash(self._config)
    
    def width(self):
        return self._width
    
    def height(self):
        return self._height
    
    def pretty_str(self):
        s = ''
        for row in self._config:
            s += ''.join('o' if v==1 else '.' for v in row) + '\n'
        return s
    
    def copy(self):
        return life_pattern(self._config)

    def apply_symmetry(self, symmetry):
        new_config = [len(row)*[None] for row in self._config]
        for i, row in enumerate(symmetry):
            for j, pair in enumerate(row):
                k, l = pair
                new_config[i][j] = self._config[k][l]
        return life_pattern(new_config)

    def step(self):
        new_config = [self._width*[0] for _ in range(self._height)]
        for i in range(self._height):
            for j in range(self._width):
                s = 0
                for di, dj in life_pattern.directions:
                    ii = (i + di) % self._height
                    jj = (j + dj) % self._width
                    s += self._config[ii][jj]
                if s == 3 or (self._config[i][j] == 1 and s == 2):
                    new_config[i][j] = 1
        return life_pattern(new_config)

A board configuration is stored as a "tuple of tuples", which we informally refer to as a *rectangle*. For example, a single glider in a $6\times 5$ board can be represented by:

    ((0,1,0,0,0),
     (0,0,1,0,0),
     (1,1,1,0,0),
     (0,0,0,0,0),
     (0,0,0,0,0))

To create a `life_board` object representing this pattern we can use:

In [9]:
glider = life_pattern(((0,1,0,0,0),
                       (0,0,1,0,0),
                       (1,1,1,0,0),
                       (0,0,0,0,0),
                       (0,0,0,0,0)))
print(glider.pretty_str())

.o...
..o..
ooo..
.....
.....



The method `pretty_str()` returns a string with a pretty representation of the board.

The evolution of the pattern can be computed as follows:

In [10]:
pattern = glider.copy()
print(pattern.pretty_str())
for i in range(4):
    pattern = pattern.step()
    print(pattern.pretty_str())

.o...
..o..
ooo..
.....
.....

.....
o.o..
.oo..
.o...
.....

.....
..o..
o.o..
.oo..
.....

.....
.o...
..oo.
.oo..
.....

.....
..o..
...o.
.ooo.
.....



Objects of type `life_pattern` are considered to be immutable. Calling the method `step()` returns a new object. For this reason, to preserve the initial state we first create a copy:

    pattern = glider.copy()
    
Then, in the loop, we compute the successive steps and print each iteration.

Not doing the computation "in place" seems to be wasteful and inefficient, but is what fits better our purposes. We will want to keep a record of the whole evolution of a pattern, so creating a new object in each step makes sense. Furthermore, we will want to store patterns in Python sets and dictionaries, so they have to be immutable.

The next function provides a better way to print the whole history of a pattern's evolution:

In [11]:
def print_history(history):
    print_list = [pattern.pretty_str() for pattern in history]
    split_list = [s.split('\n')[:-1] for s in print_list]
    for row in list(zip(*split_list)):
        print(' '.join(row))

Using this, we can print the evolution of the glider pattern as follows:

In [12]:
pattern = glider.copy()
history = [pattern]
for i in range(4):
    pattern = pattern.step()
    history.append(pattern)
print_history(history)

.o... ..... ..... ..... .....
..o.. o.o.. ..o.. .o... ..o..
ooo.. .oo.. o.o.. ..oo. ...o.
..... .o... .oo.. .oo.. .ooo.
..... ..... ..... ..... .....


The following cell shows how to extend the computation to continue the pattern evolution. Computing this cell repeately (using `<control-enter>`) yields an "animation" of the evolution, four steps at a time.

In [22]:
pattern = history[-1].copy()
history = [pattern]
for i in range(4):
    pattern = pattern.step()
    history.append(pattern)
print_history(history)

.o... ..... ..... ..... .....
..o.. o.o.. ..o.. .o... ..o..
ooo.. .oo.. o.o.. ..oo. ...o.
..... .o... .oo.. .oo.. .ooo.
..... ..... ..... ..... .....


We now consider detection of periodicities. A naive approach would be to record the whole history and check if each new pattern has already appeared. However, there is a well-known shortcut, known as *Floyd's cycle detection algorithm*, first described by Donald Knuth. For a general discussion of the algorithm and its history, see the [Wikipedia article on Floyd's algorithm](https://en.wikipedia.org/wiki/Cycle_detection)

Given a sequence successive configurations $C_0,C_1,C_2,\ldots$ Floyd's algorithm searches periodicities by looking for an index $k$ for which $C_{k}=C_{2k}$. This can be done without recording the whole evolution history, iterating both a single step of the algorithm (the *tortoise*) and two steps of the algorithm (the *hare*). The algorithm below is a direct adaptation of the code in the Wikipedia article linked above.

In [23]:
def detect_cycle(pattern):
    # Phase 1: find a repetition
    tortoise = pattern.step()
    hare = tortoise.step()
    while tortoise != hare:
        tortoise = tortoise.step()
        hare = hare.step().step()
    # Phase 2: find position of first repetition
    first_rep = 0
    tortoise = pattern.copy()
    while tortoise != hare:
        tortoise = tortoise.step()
        hare = hare.step()
        first_rep += 1
    # Phase 3: find the length of the cycle
    period = 1
    hare = tortoise.step()
    while tortoise != hare:
        hare = hare.step()
        period += 1
    return first_rep, period

In [24]:
glider = life_pattern(((0,1,0,0,0),
                       (0,0,1,0,0),
                       (1,1,1,0,0),
                       (0,0,0,0,0),
                       (0,0,0,0,0)))
first_rep, period = detect_cycle(glider)
first_rep, period

(0, 20)

This tells us that, in the $6\times 5$ board, the glider pattern repeats with a period of 20. We can verify this computing the whole history of the evolution:

In [25]:
pattern = glider.copy()
history = [pattern]
for i in range(20):
        pattern = pattern.step()
        history.append(pattern)
print_history(history[:10])
print()
print_history(history[10:])

.o... ..... ..... ..... ..... ..... ..... ..... ..... ...o.
..o.. o.o.. ..o.. .o... ..o.. ..... ..... ..... ..... .....
ooo.. .oo.. o.o.. ..oo. ...o. .o.o. ...o. ..o.. ...o. .....
..... .o... .oo.. .oo.. .ooo. ..oo. .o.o. ...oo ....o ..o.o
..... ..... ..... ..... ..... ..o.. ..oo. ..oo. ..ooo ...oo

...oo ...oo o..oo o...o o..o. oo... .o... .o..o .o... o.... .o...
..... ..... ..... ....o o...o o...o oo..o oo... .o..o .oo.. ..o..
..... ..... ..... ..... ..... ..... ..... o.... oo... oo... ooo..
....o ...o. ....o ..... ..... ..... ..... ..... ..... ..... .....
..o.o o...o o.... o..o. o.... ....o o.... ..... ..... ..... .....


We are now ready to look for oscillator in the general $3\times n$ board. We take a brute-force approach, testing for all possible initial patterns. We can however, save some computational effort by not repeating the computation for symmetric patterns. A general rectangle has 4 symmetries: the identity, horizontal reflection, vertical reflection and 180 degree rotation. If the rectangle is a square, there are a total of 8 symmetries. The following function generates the symmetries of general rectangle:

In [26]:
def rectangle_symmetries(m, n, skip_id=True):
    start_rect = [[(i,j) for j in range(n)] for i in range(m)]
    if not skip_id:
        yield start_rect
    new_rect = list([list(reversed(row)) for row in start_rect])
    yield new_rect
    new_rect = list(reversed(start_rect))
    yield new_rect
    new_rect = [list(reversed(row)) for row in new_rect]
    yield new_rect
    if m != n:
        return
    start_rect = [[(j,i) for j in range(n)] for i in range(m)]
    yield start_rect
    new_rect = [list(reversed(row)) for row in start_rect]
    yield new_rect
    new_rect = list(reversed(start_rect))
    yield new_rect
    new_rect = [list(reversed(row)) for row in new_rect]
    yield new_rect

For example, the following are the symmetries of a $3\times 5$ rectangle:

In [27]:
for symmetry in rectangle_symmetries(3, 5, False):
    for row in symmetry:
        for elem in row:
            print(elem, end=' ')
        print()
    print()

(0, 0) (0, 1) (0, 2) (0, 3) (0, 4) 
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4) 
(2, 0) (2, 1) (2, 2) (2, 3) (2, 4) 

(0, 4) (0, 3) (0, 2) (0, 1) (0, 0) 
(1, 4) (1, 3) (1, 2) (1, 1) (1, 0) 
(2, 4) (2, 3) (2, 2) (2, 1) (2, 0) 

(2, 0) (2, 1) (2, 2) (2, 3) (2, 4) 
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4) 
(0, 0) (0, 1) (0, 2) (0, 3) (0, 4) 

(2, 4) (2, 3) (2, 2) (2, 1) (2, 0) 
(1, 4) (1, 3) (1, 2) (1, 1) (1, 0) 
(0, 4) (0, 3) (0, 2) (0, 1) (0, 0) 



For a $3\times 3$ rectangle we get 8 symmetries: 

In [28]:
for symmetry in rectangle_symmetries(3, 3, False):
    for row in symmetry:
        for elem in row:
            print(elem, end=' ')
        print()
    print()

(0, 0) (0, 1) (0, 2) 
(1, 0) (1, 1) (1, 2) 
(2, 0) (2, 1) (2, 2) 

(0, 2) (0, 1) (0, 0) 
(1, 2) (1, 1) (1, 0) 
(2, 2) (2, 1) (2, 0) 

(2, 0) (2, 1) (2, 2) 
(1, 0) (1, 1) (1, 2) 
(0, 0) (0, 1) (0, 2) 

(2, 2) (2, 1) (2, 0) 
(1, 2) (1, 1) (1, 0) 
(0, 2) (0, 1) (0, 0) 

(0, 0) (1, 0) (2, 0) 
(0, 1) (1, 1) (2, 1) 
(0, 2) (1, 2) (2, 2) 

(2, 0) (1, 0) (0, 0) 
(2, 1) (1, 1) (0, 1) 
(2, 2) (1, 2) (0, 2) 

(0, 2) (1, 2) (2, 2) 
(0, 1) (1, 1) (2, 1) 
(0, 0) (1, 0) (2, 0) 

(2, 2) (1, 2) (0, 2) 
(2, 1) (1, 1) (0, 1) 
(2, 0) (1, 0) (0, 0) 



The next function searches for cycles in all possible starting cofigurations in a $3\times n$ board. Each starting configurations is stored in a Python set, so that we can avoid checking symmetric configurations. We record all oscillators found that way in a set object.

In [34]:
from itertools import product, combinations
def find_oscillators(n):
    # Generate list of all pairs (i,j) where 0<=i<3 and 0<=j<n
    pairs = list(product(range(3), range(n)))
    visited = set()
    oscillators = set()
    # The following iterated loops generates all subsets
    # of the set of pairs
    for k in range(len(pairs)+1):
        for subset in combinations(pairs, k):
            # Generate initial pattern
            config = [n*[0] for _ in range(3)]
            for i, j in subset:
                config[i][j] = 1
            pattern = life_pattern(config)
            # Check if pattern is symmetric to a pattern already visited
            if any([pattern.apply_symmetry(symmetry) in visited
                   for symmetry in rectangle_symmetries(3,n)]):
                continue
            # If this is a new pattern, add it to visited patterns
            # and check for periodicities
            visited.add(pattern)
            first_rep, period = detect_cycle(pattern)
            # Check if we have an oscillator
            # If we do, recompute the sequence to find the oscillator
            # We can reuse the pattern because we don't need it for
            # anything else
            if period > 1:
                for i in range(first_rep):
                    pattern = pattern.step()
                # Add the pattern to the set of oscillators. We also add it
                # to the set of visited patterns
                oscillators.add(pattern)
                visited.add(pattern)
    return oscillators

In [35]:
for n in range(1, 5):
    oscillators = find_oscillators(n)
    count = len(oscillators)
    print('n={}, number of oscillators found: {}'.format(n, count))
    if count > 0:
        for pattern in oscillators:
            print(pattern.pretty_str())

n=1, number of oscillators found: 0
n=2, number of oscillators found: 0
n=3, number of oscillators found: 0
n=4, number of oscillators found: 3
.oo.
.oo.
.oo.

oo..
oo..
oo..

o..o
o..o
o..o



This answers the question asked in The Riddler: the smallest value of $n$ for which there are oscillators is 4. The following cell computes the evolution of each of the oscillators:

In [38]:
oscillators = find_oscillators(4)
for pattern in oscillators:
    # Compute the period
    first_rep, period = detect_cycle(pattern)
    history = [pattern]
    for i in range(period):
        pattern = pattern.step()
        history.append(pattern)
    print_history(history)
    print()

.oo. o..o .oo.
.oo. o..o .oo.
.oo. o..o .oo.

oo.. ..oo oo..
oo.. ..oo oo..
oo.. ..oo oo..

o..o .oo. o..o
o..o .oo. o..o
o..o .oo. o..o



Notice that the pattern:

    ..oo
    ..oo
    ..oo
    
does not appear in the list of oscillator because it is symmetric to the pattern:

    oo..
    oo..
    oo..

We conclude that there is a total of 4 oscillators on a $3\times 4$ board.

We can continue the investigation, trying to find oscillators on larger boards:

In [39]:
n = 5
oscillators = find_oscillators(n)
print("Number of oscillators for n={}: {}".format(n, len(oscillators)))

Number of oscillators for n=5: 0


In [40]:
n = 6
oscillators = find_oscillators(n)
print("Number of oscillators for n={}: {}".format(n, len(oscillators)))

Number of oscillators for n=6: 0


Curiously, there are no oscillators for $n=5$ or $n=6$!

Let's try to find oscillators for $n=7$. 

**Warning:** The following cell will take a long time to compute!

In [41]:
n = 7
oscillators = find_oscillators(n)
print("Number of oscillators for n={}: {}".format(n, len(oscillators)))

Number of oscillators for n=7: 145


So, there are a lot of oscillators for $n=7$! Let's print them. To save space, let's print all the disjoint cycles that we can find.

*Notice that our code does not find all oscillators, since we avoid symmetries*. Of course, it is possible to find all the oscillators by applying all possible symmetries to the ones we found.

In [43]:
visited = set()
count = 0
for pattern in oscillators:
    # Skip patterns that already appeared in some cycle
    if pattern in visited:
        continue
    # Compute the period
    first_rep, period = detect_cycle(pattern)
    history = [pattern]
    for i in range(period):
        pattern = pattern.step()
        history.append(pattern)
    count += 1
    print('Cycle {}:'.format(count))
    print_history(history)
    # Add all patterns in history to visited
    for pattern in history:
        visited.add(pattern)
    print()    

Cycle 1:
..o..o. ..o..oo o.o.... o.o...o ..o..o.
..oo.o. ..oo.oo o.oo... o.oo..o ..oo.o.
...o.o. ...o.oo o..o... o..o..o ...o.o.

Cycle 2:
..oo.o. ..oo.oo o.oo... o.oo..o ..oo.o.
..o..o. ..o..oo o.o.... o.o...o ..o..o.
...o.o. ...o.oo o..o... o..o..o ...o.o.

Cycle 3:
o...o.. oo..o.. ..o.o.. .oo.o.. o...o..
o....o. oo...o. ..o..o. .oo..o. o....o.
o...oo. oo..oo. ..o.oo. .oo.oo. o...oo.

Cycle 4:
o.oo..o o...o.o o..oo.o o.o...o o.oo..o
o.oo..o o...o.o o..oo.o o.o...o o.oo..o
..oo... ....o.. ...oo.. ..o.... ..oo...

Cycle 5:
o.oo..o o...o.o o..oo.o o.o...o o.oo..o
..oo... ....o.. ...oo.. ..o.... ..oo...
o.oo..o o...o.o o..oo.o o.o...o o.oo..o

Cycle 6:
oo...o. ..o.oo. .oo...o ...o.oo o.oo... o...o.o .o.oo.. oo...o.
oo...o. ..o.oo. .oo...o ...o.oo o.oo... o...o.o .o.oo.. oo...o.
oo...o. ..o.oo. .oo...o ...o.oo o.oo... o...o.o .o.oo.. oo...o.

Cycle 7:
...oo.o o..oo.o .o.oo.. oo.oo.. ...oo.o
...oo.o o..oo.o .o.oo.. oo.oo.. ...oo.o
......o o.....o .o..... oo..... ......o

Cycle 8:
..o..o. .

We conclude that the patterns found break down in 45 disjoint cycles. We emphasize again that this is not an accurate count of all oscillators, due to the way our code treats symmetric patterns. It is curious to observe that all oscillators for $n=7$ have period 4 or 7.